In [22]:
import os
import SimpleITK as sitk
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

import nibabel
from monai.transforms import Resize

import time

In [23]:
import sys
sys.path.insert(1, '/workspace')

In [24]:
from_dir = '/workspace/BoneMeta_all_192/preds/bonemeta_fn_0_3D_Unet_DropRes_lv6_All192_model_epoch800/numpy'
to_dir = '/workspace/BoneMeta_all_192/preds/bonemeta_fn_0_3D_Unet_DropRes_lv6_All192_model_epoch800/pred_for_test'
image_dir = '/workspace/BoneMeta_nnUnet/images'

os.makedirs(to_dir, exist_ok=True)

data_info_path = '/workspace/DataInfo/data_info_V_2022_04_23.csv'
WHICH_SHAPE = 'Original Shape' # '1mm Resized Shape' # 

def get_label_path(file):
    return os.path.join(label_data_dir, file)

def get_image_path(file):
    return os.path.join(from_dir, file)

def case_to_file(case):
    return case+'.npy'

def file_to_case(file_name):
    return file_name.split('.')[0]

In [25]:
data_info = pd.read_csv(data_info_path)
data_info.tail()

,Data,Case,Lesion Type,Lesion Location,Probability,I/V,File Name,Lesion Size,Background Size,Original Shape,...,Original Center,1mm Resized Center,Pixel Size,Spine Shape,Spine Resized Shape,Spine Center,Spine Resized Shape.1,X Spacing,Y Spacing,Slice Interval
2654,train_add,SN051_20170401_CT_Chest_+_3D_(contrast_NO_CM),S,L1,100,V,lesionAnnot3D-005.nii.gz,471.7,3812.3,"[512, 512, 330]",...,"[276, 311, 299]","[165, 186, 299]",1312,"[256, 256, 330]","[45, 110, 299]","[76, 183, 299]","[45, 110, 299]",0.5996,0.5996,-1.0
2655,train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,C7,100,V,lesionAnnot3D-001.nii.gz,1708.3,53307.7,"[512, 512, 341]",...,"[262, 261, 13]","[167, 166, 13]",4188,"[256, 256, 341]","[39, 84, 13]","[62, 133, 13]","[39, 84, 13]",0.6387,0.6387,-1.0
2656,train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,T7,100,V,lesionAnnot3D-004.nii.gz,2893.2,44356.8,"[512, 512, 341]",...,"[258, 322, 155]","[164, 205, 155]",7093,"[256, 256, 341]","[37, 124, 155]","[58, 194, 155]","[37, 124, 155]",0.6387,0.6387,-1.0
2657,train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,T9-1,100,V,lesionAnnot3D-005.nii.gz,3801.2,121246.8,"[512, 512, 341]",...,"[262, 363, 218]","[167, 232, 218]",9319,"[256, 256, 341]","[39, 150, 218]","[62, 235, 218]","[39, 150, 218]",0.6387,0.6387,-1.0
2658,train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,Sternum,100,V,lesionAnnot3D-006.nii.gz,16.4,300411.6,"[512, 512, 341]",...,"[298, 168, 87]","[190, 107, 87]",40313,"[256, 256, 341]","[62, 25, 87]","[98, 40, 87]","[62, 25, 87]",0.6387,0.6387,-1.0


In [26]:
data_info

,Data,Case,Lesion Type,Lesion Location,Probability,I/V,File Name,Lesion Size,Background Size,Original Shape,...,Original Center,1mm Resized Center,Pixel Size,Spine Shape,Spine Resized Shape,Spine Center,Spine Resized Shape.1,X Spacing,Y Spacing,Slice Interval
0,val,000232_20190401_CT_Abdomen+Pelvis_Dynamic_(con...,S,T10-1,metastasis,V,lesionAnnot3D-000.nii.gz,731.0,7589.0,"[512, 512, 435]",...,"[224, 298, 42]","[118, 158, 42]",2590,"[256, 256, 435]","[12, 90, 42]","[24, 170, 42]","[12, 90, 42]",0.5312,0.5312,-1.0
1,val,000232_20190401_CT_Abdomen+Pelvis_Dynamic_(con...,S,T11,metastasis,V,lesionAnnot3D-002.nii.gz,1635.2,12690.8,"[512, 512, 435]",...,"[244, 341, 65]","[129, 181, 65]",5794,"[256, 256, 435]","[23, 113, 65]","[44, 213, 65]","[23, 113, 65]",0.5312,0.5312,-1.0
2,val,000232_20190401_CT_Abdomen+Pelvis_Dynamic_(con...,S,T12,metastasis,V,lesionAnnot3D-003.nii.gz,570.1,8361.9,"[512, 512, 435]",...,"[230, 293, 83]","[122, 155, 83]",2020,"[256, 256, 435]","[16, 87, 83]","[30, 165, 83]","[16, 87, 83]",0.5312,0.5312,-1.0
3,val,000232_20190401_CT_Abdomen+Pelvis_Dynamic_(con...,S,L1-1,metastasis,V,lesionAnnot3D-004.nii.gz,3222.2,54917.8,"[512, 512, 435]",...,"[262, 282, 117]","[139, 149, 117]",11417,"[256, 256, 435]","[33, 81, 117]","[62, 154, 117]","[33, 81, 117]",0.5312,0.5312,-1.0
4,val,000232_20190401_CT_Abdomen+Pelvis_Dynamic_(con...,S,L1-2,metastasis,V,lesionAnnot3D-005.nii.gz,921.8,10446.2,"[512, 512, 435]",...,"[273, 273, 128]","[145, 145, 128]",3266,"[256, 256, 435]","[39, 77, 128]","[73, 145, 128]","[39, 77, 128]",0.5312,0.5312,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2654,train_add,SN051_20170401_CT_Chest_+_3D_(contrast_NO_CM),S,L1,100,V,lesionAnnot3D-005.nii.gz,471.7,3812.3,"[512, 512, 330]",...,"[276, 311, 299]","[165, 186, 299]",1312,"[256, 256, 330]","[45, 110, 299]","[76, 183, 299]","[45, 110, 299]",0.5996,0.5996,-1.0
2655,train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,C7,100,V,lesionAnnot3D-001.nii.gz,1708.3,53307.7,"[512, 512, 341]",...,"[262, 261, 13]","[167, 166, 13]",4188,"[256, 256, 341]","[39, 84, 13]","[62, 133, 13]","[39, 84, 13]",0.6387,0.6387,-1.0
2656,train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,T7,100,V,lesionAnnot3D-004.nii.gz,2893.2,44356.8,"[512, 512, 341]",...,"[258, 322, 155]","[164, 205, 155]",7093,"[256, 256, 341]","[37, 124, 155]","[58, 194, 155]","[37, 124, 155]",0.6387,0.6387,-1.0
2657,train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,T9-1,100,V,lesionAnnot3D-005.nii.gz,3801.2,121246.8,"[512, 512, 341]",...,"[262, 363, 218]","[167, 232, 218]",9319,"[256, 256, 341]","[39, 150, 218]","[62, 235, 218]","[39, 150, 218]",0.6387,0.6387,-1.0


In [27]:
SIZE_INFO_DF = data_info.loc[:,['Case', WHICH_SHAPE]]
SIZE_INFO_DF.drop_duplicates(inplace=True)
SIZE_INFO_DF.set_index('Case', inplace=True)
print(SIZE_INFO_DF.shape)

(318, 1)


In [28]:
SIZE_INFO_DF.head()

,Original Shape
Case,
000232_20190401_CT_Abdomen+Pelvis_Dynamic_(contrast),"[512, 512, 435]"
000232_20190423_Chest_CT_(contrast),"[512, 512, 315]"
000285_20190801_CT_Abdomen+Pelvis_Pre-Post_(contrast),"[512, 512, 528]"
000285_20190807_Chest_CT_(contrast),"[512, 512, 238]"
000316_20190627_Chest_CT_(contrast),"[512, 512, 283]"


In [29]:
def str_to_crz_list(resolution):
    crz = resolution.strip('[]').split(', ')
    return [float(value) for value in crz]

In [30]:
def save_image(np_image, target_dir, file_name):
    np_image = np_image.astype(np.int16)
    np.save(os.path.join(target_dir, file_name), np_image)
    
def save_label(np_image, target_dir, file_name):
    np_image = (np_image>0).astype(np.uint8)
    np.save(os.path.join(target_dir, file_name), np_image)

In [31]:
def str_to_size(size_str):
    crz = size_str.strip('[]').split(', ')
    torch_size = [int(value) for value in crz] 
    torch_size.reverse() # crz to zrc
    return torch_size

def get_size(file):
    case = file_to_case(file)
    size_str = SIZE_INFO_DF.loc[case][WHICH_SHAPE]
    if not type(size_str) == str:
        size_str = size_str[0]
    tensor_size = str_to_size(size_str)
    return tensor_size

def resample_all_1mm(np_image, target_size):
    resizer = Resize(target_size)
    resized_image = resizer(np_image[np.newaxis,:]) # with channel
    return resized_image[0] # without channel

def resample_all_1mm_label(np_label, target_size):
    np_label = np_label > 0.5
    resizer = Resize(target_size)
    resized_label = resizer(np_label[np.newaxis,:]) # with channel
    return resized_label[0] # without channel

In [32]:
from multiprocessing import Pool

def save_image(np_image, target_dir, file_name):
    np_image = np_image.astype(np.int16)
    np.save(os.path.join(target_dir, file_name), np_image)
    
def save_label(np_image, target_dir, file_name):
    np_image = (np_image>0.5).astype(np.uint8)
    np.save(os.path.join(target_dir, file_name), np_image)

def resample(file, file_dir, target_dir, resample_func, save_func):
    np_file = np.load(os.path.join(file_dir, file))
    target_size = get_size(file)
    resampled_image = resample_func(np_file, target_size)
    save_func(resampled_image, target_dir, file)

def wrapper(args):
    try:
        return resample(*args)
    except: 
        print(f'Fail case: {args[0]}')
        return None

t = time.perf_counter()

file_list = os.listdir(from_dir)
arg_list = [(file, from_dir, to_dir, resample_all_1mm, save_image) for file in file_list]
pool = Pool(processes=32)
pool.map(wrapper, arg_list)
pool.close()
pool.join()

elapsed_time = time.perf_counter() - t

print(f'''
consumed_time: {elapsed_time}s
''')


consumed_time: 62.122652660822496s



In [33]:
file_names = os.listdir(to_dir)
resampled_sizes = [np.load(os.path.join(to_dir, file)).shape for file in file_names]
image_sizes = [np.load(os.path.join(image_dir, file)).shape for file in file_names]

In [13]:
image_sizes == resampled_sizes

False